<h1 style='background: black; border:1; color: white'><center>Covid-19 Vaccination Drive</center></h1>

<h2 style='background: black; border:1; color: white'><center>Acknowledgement</center></h2>

This Notebook would not have been possible without the dataset provided by [@Gabriel Preda](https://www.kaggle.com/gpreda). If you haven't already, I strongly encourage you to upvote his dataset on [Covid-19 world vaccination progress](https://www.kaggle.com/gpreda/covid-world-vaccination-progress).



<h2 style='background: black; border:1; color: white'><center>Basic features of the data set</center></h2>

This is a basic exploratory data analysis of the Global Covid-19 vaccination drive. Hopefully, we can answer some interesting questions about how things are going and when we can expect things to return to "normal".

* This data set starts on 2020-12-13, which means the vaccination according to it starts on this specific date and captures the data upto 2021-02-06.

* Till date the vaccination drive is going around the world in 78 different countries with more than 10 types of vaccines are being used. 



<h2 style='background: black; border:1; color: white'><center>Table of contents </center></h2>


* [Import](#chapter1)
* [Data Cleaning](#chapter2)
    * [Some features](#chapter3)
* [Data visualization](#chapter4)
    * [Word Art of countries](#chapter5)
* [Total vaccinated till date](#chapter6)
* [Country wise daily vaccination](#chapter7)
* [Plot till date Function](#chapter23)
    * [People vaccinated vs people fully vaccinated in the world](#chapter24)
    * [ People vaccinated vs people fully vaccinated per hundred in the world](#chapter25)
* [Function for Pie-Plot](#chapter20)
    * [Most used vaccine](#chapter8)
    * [Most frequent Sources](#chapter9)
    * [Word art of Vaccines](#chapter10)
* [Daily vaccination trend per million](#chapter11)
* [Total vaccinated - India vs USA](#chapter12)
* [Function for Maps](#chapter21)
    * [Most vaccinated country](#chapter13)
    * [Vaccines Used in Different countries](#chapter14)
    * [People fully vaccinated in Different countries](#chapter15)
* [Function for Animation](#chapter22)
    * [Animation of daily vaccinations through January 2021 in different countries](#chapter16)
    * [Animation of daily vaccinations per million through January 2021 in different countries](#chapter17)
* [Date vs People Vaccinated vs People Fully Vaccinated | 3D](#chapter18)
* [Corelation Heatmap](#chapter19)

# Import : <a class="anchor" id="chapter1"></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.patches as mpatches
from plotly.subplots import make_subplots
from wordcloud import WordCloud
import seaborn as sns  
sns.set(color_codes = True)
sns.set(style="whitegrid")
import plotly.figure_factory as ff
from plotly.colors import n_colors

In [ ]:
df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

As you can see, a lot of null values are present in the data. The probable reasons would be:

1. The vaccination drive has just started. So, it hasn't quite taken pace all over the world.

2. The sources are not enough to cover the vaccination daily everywhere.

Whatever the reason might be, lets move ahead with cleaning the data and filling the missing values...

# Data cleaning : <a class="anchor" id="chapter2"></a>

In this section we have done particularly few things : 

1. Filled the null values with 0 and assumed that nothing has happened* regarding that particular column.
2. Converted all the numeric values* from object type to integer type.
3. Added 3 new columns viz **'year'**, **'month'** and **'date'**. (Asking why? You will get that later on...)
4. After adding them I turned them into integer type again and 
5. Converted the date column into datetime from object type.


In [ ]:
df.fillna(value = 0, inplace = True)
df.total_vaccinations = df.total_vaccinations.astype(int)
df.people_vaccinated = df.people_vaccinated.astype(int)
df.people_fully_vaccinated = df.people_fully_vaccinated.astype(int)

df.daily_vaccinations_raw = df.daily_vaccinations_raw.astype(int)
df.daily_vaccinations = df.daily_vaccinations.astype(int)
df.total_vaccinations_per_hundred = df.total_vaccinations_per_hundred.astype(int)

df.people_fully_vaccinated_per_hundred = df.people_fully_vaccinated_per_hundred.astype(int)
df.daily_vaccinations_per_million = df.daily_vaccinations_per_million.astype(int)
df.people_vaccinated_per_hundred = df.people_vaccinated_per_hundred.astype(int)

In [ ]:
date = df.date.str.split('-', expand =True)
date 

In [ ]:
df['year'] = date[0]
df['month'] = date[1]
df['day'] = date[2]

df.year = pd.to_numeric(df.year)
df.month = pd.to_numeric(df.month)
df.day = pd.to_numeric(df.day)

In [ ]:
df.date = pd.to_datetime(df.date)

df.head()

### DataFrame is ready to get visualized 

## Some features : <a class="anchor" id="chapter3"></a>

In [ ]:
print('Data point starts from ',df.date.min(),'\n')
print('Data point ends at ',df.date.max(),'\n')
print('Total no of countries in the data set ',len(df.country.unique()),'\n')
print('Total no of unique vaccines in the data set ',len(df.vaccines.unique()),'\n')

In [ ]:
df.info()

# Data Visualization : <a class="anchor" id="chapter4"></a>

In [ ]:
df.describe()

In [ ]:
df.country.unique()

This is a very useful sweet little **function** to resize the **Seaborn** plots

In [ ]:
def size(m,n):
    fig = plt.gcf();
    fig.set_size_inches(m,n);

## Word Art of countries : <a class="anchor" id="chapter5"></a>

In [ ]:
wordCloud = WordCloud(
    background_color='white',
    max_font_size = 50).generate(' '.join(df.country))

plt.figure(figsize=(15,7))
plt.axis('off')
plt.imshow(wordCloud)
plt.show()

# Total vaccinated till date : <a class="anchor" id="chapter6"></a>

In [ ]:
country_wise_total_vaccinated = {}

for country in df.country.unique() : 
    vaccinated = 0

    for i in range(len(df)) : 
        if df.country[i] == country : 
            vaccinated += df.daily_vaccinations[i]
    
    country_wise_total_vaccinated[country] = vaccinated 
#     made a seperate dict from the df 
    country_wise_total_vaccinated_df = pd.DataFrame.from_dict(country_wise_total_vaccinated,
                                                         orient='index',
                                                         columns = ['total_vaccinted_till_date'])
#     converted dict to df 
country_wise_total_vaccinated_df.sort_values(by = 'total_vaccinted_till_date', ascending = False, inplace = True)

country_wise_total_vaccinated_df

In [ ]:
fig = px.bar(country_wise_total_vaccinated_df, 
             y = 'total_vaccinted_till_date',
             x = country_wise_total_vaccinated_df.index,
             color = 'total_vaccinted_till_date',
             color_discrete_sequence= px.colors.sequential.Viridis_r
            )

fig.update_layout(
    title={
            'text' : "Vaccination till date in various countries",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Countries",
    yaxis_title="Total vaccinated",
    legend_title="Total vaccinated"
)

fig.show()

From the plot, some interesting facts are easily noticeable:

1. The **United States**, despite all protests, has the highest number of vaccines.
2. **China**, from where the virus started spreading, is at second.
3. **UK**, where we have found a new variant strain of the virus, is right next.
4. **India**, who has been supplying vaccines to the world is herself at 6th position.
5. Following that, we have **Turkey**, **UAE**, **Germany** and others.

# Country wise daily vaccination : <a class="anchor" id="chapter7"></a>

In [ ]:
fig = px.line(df, x = 'date', y ='daily_vaccinations', color = 'country')

fig.update_layout(
    title={
            'text' : "Daily vaccination trend",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Daily Vaccinations"
)

fig.show()

From this visualization we can see that :

1. **China** started **"Mass Vaccination"** first
2. Though **USA** started second, in terms of daily vaccinations it stands at the top now.
3. Following that we have **United Kingdom** and **England**
4. And then **India** comes in the 5th position in terms of daily vaccination




# Plot till date Function : <a class="anchor" id="chapter23"></a>

In [ ]:
# helper function
def plot_till_date(value1, value2, title, color1, color2) : 
    
    so_far_dict = {}
    
    for dates in df.date.unique() : 
        so_far_dict[dates], value1_count, value2_count = [], 0, 0
    
        for i in range(len(df)) : 
        
            if df.date[i] == dates : 
                value1_count += df[value1][i]
                value2_count += df[value2][i]
#         if dates not in so_far_dict.keys() : 
        
        so_far_dict[dates].append(value1_count) 
        so_far_dict[dates].append(value2_count)
    
    so_far_df = pd.DataFrame.from_dict(so_far_dict, orient = 'index', columns=[value1, value2])
    so_far_df.reset_index(inplace = True)
#     return so_far_df
    
    so_far_df.sort_values(by='index', inplace = True)
    
    plot = go.Figure(data=[go.Scatter( 
            x = so_far_df['index'], 
            y = so_far_df[value1], 
            stackgroup='one', 
            name = value1, 
            marker_color= color1), 
                               
                           go.Scatter( 
            x = so_far_df['index'], 
            y = so_far_df[value2], 
            stackgroup='one',
            name = value2, 
            marker_color= color2) 
        ]) 
    
    plot.update_layout(
    title={
            'text' : title,
            'y':0.95,
            'x':0.5
        },
        xaxis_title="Date"
    )
    
    return plot.show()

## People vaccinated vs people fully vaccinated in the world : <a class="anchor" id="chapter24"></a>

In [ ]:
plot_till_date('people_fully_vaccinated', 'people_vaccinated','People vaccinated vs Fully vaccinated till date', '#c4eb28', '#35eb28')

## People vaccinated vs people fully vaccinated per hundred in the world : <a class="anchor" id="chapter25"></a>

In [ ]:
plot_till_date('people_fully_vaccinated_per_hundred', 'people_vaccinated_per_hundred', 'People vaccinated vs Fully vaccinated per hundred till date', '#0938e3','#7127cc')

# Function for Pie-Plot : <a class="anchor" id="chapter20"></a>

In [ ]:
def plot_pie(value, title, color) : 
    
    new_dict = {}
    
    for v in df[value].unique() : 
        value_count = 0
        
        for i in range(len(df)) : 
            if df[value][i] == v : 
                value_count += 1 
        
        new_dict[v] = value_count 
        
#     print(new_dict)


    new_df = pd.DataFrame.from_dict(new_dict, orient = 'index', columns = ['Total'])
    
    if color == 'plasma' : 
        
        fig = px.pie(new_df, values= 'Total',
                     names = new_df.index,
                     title = title,
                     color_discrete_sequence=px.colors.sequential.Plasma)
    
    elif color == 'rainbow' : 
        
        fig = px.pie(new_df, values= 'Total',
                     names = new_df.index,
                     title = title,
                     color_discrete_sequence=px.colors.sequential.Rainbow)
    
    else :
        
        fig = px.pie(new_df, values= 'Total',
                     names = new_df.index,
                     title = title)
    
    fig.update_layout(
        title={
            'y':0.95,
            'x':0.5
        },
        legend_title = value
    )
        
    return fig.show()

## Most used vaccine : <a class="anchor" id="chapter8"></a>


In [ ]:
df.vaccines.unique()

In [ ]:
print(len(df.vaccines.unique()))

**20 unique vaccines being distributed currently**

In [ ]:
plot_pie('vaccines', 'Various vaccines and their uses', 'plasma')

From this Pie chart we can see that : 

1. The most used vaccine around the world is **Pfizer/BioNTech** which constitutes around **47.6%** of the totals vaccines used.
2. And the second most used vaccine is **Moderna** 
3. Then comes **Oxford/AstraZeneca, Sinovac** , **Pfizer/BioNTech, Sinopharm** and **Sputnik V**
4. **Indian** **Covaxin, Oxford/AstraZeneca** comes at 10th with **0.933%** market share.

# Most frequent Sources : <a class="anchor" id="chapter9"></a>


In [ ]:
plot_pie('source_name', 'Sources', 'rainbow')

### Correction : 
Initially there were **36** sources. Now there is **40+** sources that's why the shares are getting devided.


## Word art of Vaccines<a class="anchor" id="chapter10"></a>

In [ ]:
wordCloud = WordCloud(
    background_color='white',
    max_font_size = 50).generate(' '.join(df.vaccines))

plt.figure(figsize=(12,5))
plt.axis('off')
plt.imshow(wordCloud)
plt.show()

# Daily vaccination trend per million : <a class="anchor" id="chapter11"></a>

In [ ]:
fig = px.line(df, x = 'date', y ='daily_vaccinations_per_million', color = 'country')

fig.update_layout(
    title={
            'text' : "Daily vaccination trend per million",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Daily Vaccinations per million"
)

fig.show()

Currently **Israel**, **Seychelles**, **Gibraltar**, **UAE** are leading the charge in Vaccination Rates

# Total vaccinated - China vs USA : <a class="anchor" id="chapter12"></a>

In [ ]:
china_usa = [df[df.country == 'United States'], df[df.country == 'China']]
result = pd.concat(china_usa)

fig = px.line(result, x = 'date', y ='total_vaccinations', color = 'country')

fig.update_layout(
    title={
            'text' : "Total vaccinated - China vs USA",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Total Vaccinations"
)

fig.show()

From the comparison between **China** and **USA** we can see that 

1. **USA** started the vaccination drive relative earlier.
2. both are increasingly steadily at a similar rate

# Function for Maps : <a class="anchor" id="chapter21"></a>

In [ ]:
# helper function
def plot_map(variable, title, color) : 
    
    new_dict = {}
    
    for country in df.country.unique() : 
        
        for i in range(len(df)) : 
            if df.country[i] == country : 
                new_dict[country] = df[variable][i]
    
    new_dict
    
    new_df = pd.DataFrame.from_dict(new_dict, orient='index', columns =[variable])
    
    if color == None : 
        
        place_map = px.choropleth(new_df, locations =new_df.index,
                                  locationmode = 'country names',
                                  color=variable, 
                                  hover_name= new_df.index)
    
    else : 
        
        place_map = px.choropleth(new_df, locations =new_df.index,
                                  locationmode = 'country names',
                                  color=variable, 
                                  hover_name= new_df.index,
                                  color_continuous_scale= color)
        
    place_map.update_layout(
        title_text = title,
        title_x = 0.5,
        geo=dict(showocean=True, oceancolor="#7af8ff",
                showland=True, landcolor="white",
                showframe = False))
    
    
    return place_map.show()

## Most vaccinated country : <a class="anchor" id="chapter13"></a>

In [ ]:
plot_map('total_vaccinations','Most vaccinated country', None)

## Vaccines Used in Different countries : <a class="anchor" id="chapter14"></a>

In [ ]:
plot_map('vaccines','Vaccines Used in Different countries', None)

From this plot you can see the use of vaccines around the world : 

1. Though **India** sent it's vaccine to **Brazil**, it seems that Brazil has used the **Oxford** vaccine so far. Maybe in future we will get to see more use of **Covaxine** in Brazil.
2. In **North America** and **Europe**, **Pfizer** and **Moderna** are used widely.
3. In **Asia, Africa** and **South America**, **"Sputnik V"** is being used mostly. 

## People fully vaccinated in Different countries : <a class="anchor" id="chapter15"></a>

In [ ]:
plot_map('people_fully_vaccinated','People fully vaccinated in Different countries', 'haline')

This is quite an eye opener to see that 

* Though the vaccination drive is going in full pace in most of the countries people are not fully vaccinated 

# Function for Animation: <a class="anchor" id="chapter22"></a>

## Animation of daily vaccinations through January 2021 in different countries: <a class="anchor" id="chapter16"></a>

In [ ]:
def plot_animation(value, title, color) : 
    
    new_df = df[df.year == 2021]
    
    new_df.sort_values(by='day', inplace = True)
    
    if color == None : 
        
        place_map = px.choropleth(new_df, locations ="country",
                                    locationmode = "country names", 
                                    color=value,
                                    hover_name="country",
                                    animation_frame="day")
    
    else : 
        
        place_map = px.choropleth(new_df, locations ="country",
                                locationmode = "country names", 
                                color=value,
                                hover_name="country",
                                animation_frame="day",
                                color_continuous_scale= color)

    place_map.update_layout(
        title_text = title,
        title_x = 0.5,
        geo=dict(showocean=True, oceancolor="#7af8ff",
                showland=True, landcolor="white",
                showframe = False))
    
    return place_map.show()


In [ ]:
plot_animation('daily_vaccinations', 'Animation of daily vaccinations through January 2021 in different countries', None)

## Animation of daily vaccinations per million through January 2021 in different countries: <a class="anchor" id="chapter17"></a>

In [ ]:
plot_animation('daily_vaccinations_per_million', 'Animation of daily vaccinations per million through January 2021 in different countries', 'Plotly3')

# Date vs People Vaccinated vs People Fully Vaccinated | 3D: <a class="anchor" id="chapter18"></a>

In [ ]:
fig = px.scatter_3d(df, x='date', y='people_vaccinated', z='people_fully_vaccinated',
                    color='country',
                    hover_data=['country'],
                    size = 'people_fully_vaccinated',
                    opacity=0.9, 
                    symbol = 'country')


fig.update_layout(title='Date vs People Vaccinated vs People Fully Vaccinated | 3D')

fig.update_layout(
        title={
            'y':0.95,
            'x':0.5
        }
    )

fig.show()

# Corelation Heatmap: <a class="anchor" id="chapter19"></a>

In [ ]:
sns.heatmap(df.corr())
plt.title('Corelation Heatmap')
plt.yticks(rotation = 0);
size(10,7)

<h2 style='background: black; border:1; color: white'><center>Conclusion</center></h2>

Vaccination is well underway but globally, most are not vaccinated.


<h2 style='background: black; border:1; color: white'><center>Some Key observations</center></h2>

2. In **North America** and **Europe Pfizer** and **Moderna** is widely used 
3. In **Asia, Africa** and **South America Sputnik V** is mostly used 
4. The most used vaccine around the world is **Pfizer/ BioNTech**. Around **47.6%** of the total vaccines used 

